In [1]:
# from Crypto.Hash import keccak

# def keccak256(string: str) -> str:
#     new_cal = keccak.new(digest_bits=256)
#     return new_cal.update(string.encode()).hexdigest()

# keccak256('123')

In [13]:
import json
from mimc import Zero, FieldNumber as FN, MiMCSponge

series_dct = json.load(open('./dct-serialize.json'))

assert bin(len(series_dct) + 1)[2:].count('1') == 1, "Length doesn't match"
assert [l[0] for l in series_dct] == list(range(len(series_dct))), "Index doesn't match"

mimc = MiMCSponge(-1, 220, -1)

node_dict = {}
for k in range(len(series_dct)-1, -1, -1):
    node_info = series_dct[k]
    
    if node_info[1] == -1:
        node_hash = mimc.run_nto1([
            FN(node_info[2]),           # Leaf class
            FN(node_info[0]),           # Leaf location
        ])
    else:
        node_hash = mimc.run_nto1([
            FN(node_dict[k*2+1]),       # Left child hash
            FN(node_dict[k*2+2]),       # Right child hash
            FN(node_info[1]),           # Node attribute
            FN(node_info[2]),           # Node threshold
        ])
    
    node_dict[k] = node_hash.number

In [34]:
select_leaf = 46

input_node_hash = []
while select_leaf > 0:
    neighbor = (select_leaf + 1) if select_leaf & 1 else (select_leaf - 1)
    select_leaf  = (select_leaf - 1) // 2
    input_node_hash.append((neighbor, node_dict[neighbor]))

for o in ['"' + str(o[1]) + '",' for o in input_node_hash]:
    print(o)

"14738284516118542279044613062392077453672527800542877369084326368998413463534",
"6707137214048112169348076733962540460483072835500493119566725794349053838632",
"10023264508974121377686209219543330760520721132774232588247641165170348552157",
"13005730435519623420939719315271189808619950510438419779308096714144661555894",
"6525725329685632679013351469105524719410394627460605249044401523430536622411",


In [39]:
randomness = 11111111111111111111111111111111111111111111111111111111111111

mimc.run_nto1([
    FN(node_dict[0]),           # Original root hash
    FN(randomness),             # Randomness
]).number

2247812535922842444482633518805301837146133301323337024132694587022628543208